In [1]:
import os
os.cpu_count()

8

import time
from datetime import timedelta

start_time = time.time()
for i in range(10000):
    for k in range(i):
        m=k+i
elapsed_time_secs = time.time() - start_time
msg = "Execution took: %s secs (Wall clock time)" % timedelta(seconds=round(elapsed_time_secs))
msg

In [2]:
from IPython.display import clear_output 
for kk in range(0,35,5):
    for i in range(0,5): 
        clear_output() 
        print(i+kk)

34


'p_values' : range(0, 12),#[0, 1, 4, 6, 8, 10],
'd_values' : range(0, 2),#[0, 1, 2], #range(0, 3)
'q_values' : range(0, 12),#[0, 1, 2], #range(0, 3)
'sp_values': range(0, 12),#[0, 1, 4, 6, 8, 10],
'sd_values': range(0, 2),#[0, 1, 2],
'sq_values': range(0, 12) #[0, 1, 2]

In [3]:
file_name2    = 'score_table_SARIMAX2.csv'
file_name    = 'score_table_SARIMAX.csv'
file_network = 'network_table_SARIMAX.csv'

In [4]:
import pandas as pd
from IPython.display import clear_output 
import time
from datetime import timedelta

#score_table
generations  = [10, 20, 50, 100, 200]  # Number of times to evole the population.
populations  = [10, 50, 100, 200, 300, 400, 500]  # Number of networks in each generation.

score_table = pd.DataFrame(index=range(0,175),columns= ['generations', 'population', '(p,d,q)', '(P,D,Q)','AIC','Time'])

k=0
for i in generations: 
    for j in populations:
        #print(i,j)
        l=k+4
        score_table.loc[k:l,:2] = (i, j)
        k=k+5

#score_table.set_index(['generations', 'population'])    
#score_table
def return_pos(gen,pop):
     return score_table.index[(score_table['generations'] == gen) & (score_table['population'] == pop)][0]
    

# parallel code

In [5]:
"""Entry point to evolving the neural network. Start here."""
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed

from __future__ import print_function
import logging
from optimizer import Optimizer
from tqdm import tqdm
import sys

# Setup logging.
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO#,
    #filename='log.txt'
)

def train_networks(networks, dataset, type_ser):
    """Train each network.

    Args:
        networks (list): Current population of networks
        dataset (str): Dataset to use for training/evaluating
    """
    pbar = tqdm(total=len(networks))
    for network in networks:
        network.train(dataset, type_ser)
        pbar.update(1)
    pbar.close()

def get_average_accuracy(networks):
    """Get the average accuracy for a group of networks.

    Args:
        networks (list): List of networks

    Returns:
        float: The average accuracy of a population of networks.

    """
    total_accuracy = 0
    for network in networks:
        if network.accuracy != -1 and network.accuracy != None:
            total_accuracy += network.accuracy
        else:
            continue

    return total_accuracy / len(networks)

def get_min_mse(networks):
    """Get the average accuracy for a group of networks/genomes.

    Args:
        networks (list): List of networks/genomes

    Returns:
        float: The average accuracy of a population of networks/genomes.

    """
    min_accuracy = 1000000
    min_network  = None
    for network in networks:
        if network.accuracy != -1 and network.accuracy < min_accuracy:
            min_accuracy = network.accuracy
            min_network=network
        else:
            continue
    return min_accuracy,min_network

def generate(cfg, k, nn_param_choices, dataset, type_ser):
    start_time = time.time()
    generations, population = cfg
    """Generate a network with the genetic algorithm.

    Args:
        generations (int): Number of times to evole the population
        population (int): Number of networks in each generation
        nn_param_choices (dict): Parameter choices for networks
        dataset (str): Dataset to use for training/evaluating

    """
    logging.info("***Evolving %d generations with population %d***" % (generations, population))
    logging.info("***generate(generations, population, nn_param_choices, dataset)***")
    optimizer = Optimizer(nn_param_choices)
    #print("--1--")
    networks = optimizer.create_population(population)
    #print("--2--")
    min_networks=[]
    # Evolve the generation.
    for i in range(generations):
        logging.info("*** Now in generation %d of %d ***" %(i + 1, generations))
        #print_networks(networks)
        # Train and get accuracy for networks.
        train_networks(networks, dataset, type_ser)

        # Get the average accuracy for this generation.
        average_accuracy = get_average_accuracy(networks)

        min_mse,net =get_min_mse(networks)
        #print_networks(net)
        #net.print_network()
        # Print out the average accuracy each generation.
        logging.info("Generation average: %.2f" % (average_accuracy ))#* 100
        logging.info('-'*80) #-----------
        logging.info("Generation min_mse: %.2f" % (min_mse ))#* 100
        logging.info('-'*80) #-----------

        # Evolve, except on the last iteration.
        if i != generations - 1:
            # Do the evolution.
            networks = optimizer.evolve(networks)
        min_networks.append(net)
        clear_output()
    # Sort our final population.
    # Sort our final population according to performance.
    networks=[x for x in networks if x.accuracy !=-1 and x.accuracy != None]
    networks = sorted(networks, key=lambda x: x.accuracy, reverse=False)
    print("Generation done")

    # Print out the top 5 networks.
    logging.info("top 5 networks Generation" )
    logging.info('='*80) #-----------
    print_networks(networks[:5])
    
    elapsed_time_secs = time.time() - start_time
    #pd.DataFrame(networks[:5]).to_csv(file_network, sep=',', encoding='utf-8', mode='a', header=True)
    for n in range(0,5):
        pos=return_pos(generations, population)
        score_table.loc[pos+n,2:] = networks[n].param, networks[n].param_seasonal, networks[n].accuracy, elapsed_time_secs#timedelta(seconds=round(elapsed_time_secs))
        #score_table.loc[pos+n,0] = timedelta(seconds=round(elapsed_time_secs))
    score_table.to_csv(file_name, sep=',', encoding='utf-8', mode='a', header=True)
    
    logging.info("min networks Generation" )
    logging.info('-'*80) #-----------
    min_networks = sorted(min_networks, key=lambda x: x.accuracy, reverse=False)
    print_networks(min_networks)
    return networks
    
def print_networks(networks):
    """Print a list of networks.

    Args:
        networks (list): The population of networks

    """
    logging.info('#'*80)
    for network in networks:
        network.print_network()
        
# grid search configs
def grid_search(dataset, models_gen, nn_param_choices, type_ser, parallel=True):
	scores = None
	k=0
	if parallel:
    #print("$$$$$$$$$$$$$$$$$$$$ %s $$$$$$$$$$$$$$$$$"% (type_ser))
    #networks = generate(generation, population, k, nn_param_choices, dataset, type_ser)
    #k=k+5
		# execute configs in parallel cfg, k, nn_param_choices, dataset, type_ser
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		#tasks = (delayed(generate)(generation, population, k, nn_param_choices, dataset, type_ser) for cfg in cfg_list)
		networks = (delayed(generate)(cfg, k, nn_param_choices, dataset, type_ser) for cfg in models_gen)
		scores   = executor(networks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	#scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	#scores.sort(key=lambda tup: tup[1])
	return scores
    
def main():
    """Evolve a network."""
    generations  = [100,200]  # Number of times to evole the population.[10, 20, 50, 100, 200]
    populations  = [ 300, 400, 500]  # Number of networks in each generation. [10, 50, 100, 200, 300, 400, 500]
    dataset = 'cifar10'
   
    nn_param_choices = {
        'p_values' : range(0, 13),#[0, 1, 4, 6, 8, 10],
        'd_values' : range(0, 2),#[0, 1, 2], #range(0, 3)
        'q_values' : range(0, 13),#[0, 1, 2], #range(0, 3)
        'sp_values': range(0, 13),#[0, 1, 4, 6, 8, 10],
        'sd_values': range(0, 2),#[0, 1, 2],
        'sq_values': range(0, 13) #[0, 1, 2]
    }
    type_ser ='normalized'
    models_gen = list()
    #for type_ser in ['normal', 'log', 'loglog']: #'normalized',
    for generation in generations: #'normalized',
        for population in populations:
            cfg1 = [generation, population]
            models_gen.append(cfg1)
          
    networks = grid_search(dataset, models_gen, nn_param_choices, type_ser)       
    return networks

/usr/local/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
#if __name__ == '__main__':
networks = main()


05/15/2019 11:21:17 PM - INFO - ***Evolving 200 generations with population 400***
05/15/2019 11:21:17 PM - INFO - ***Evolving 100 generations with population 400***
05/15/2019 11:21:17 PM - INFO - ***Evolving 100 generations with population 500***
05/15/2019 11:21:17 PM - INFO - ***Evolving 100 generations with population 300***
05/15/2019 11:21:17 PM - INFO - ***Evolving 200 generations with population 500***
05/15/2019 11:21:17 PM - INFO - ***Evolving 200 generations with population 300***
05/15/2019 11:21:17 PM - INFO - ***generate(generations, population, nn_param_choices, dataset)***
05/15/2019 11:21:17 PM - INFO - ***generate(generations, population, nn_param_choices, dataset)***
05/15/2019 11:21:17 PM - INFO - ***generate(generations, population, nn_param_choices, dataset)***
05/15/2019 11:21:17 PM - INFO - ***generate(generations, population, nn_param_choices, dataset)***
05/15/2019 11:21:17 PM - INFO - ***generate(generations, population, nn_param_choices, dataset)***
05/15/2

05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:17 PM - INFO - add_network() ERROR: hash clash - duplicate 

05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate 

05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate 

05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate network
05/15/2019 11:21:18 PM - INFO - add_network() ERROR: hash clash - duplicate 

...

  0%|          | 1/500 [05:09<42:56:42, 309.82s/it]05/15/2019 11:26:35 PM - INFO - SARIMAX: (7, 0, 8) x (6, 1, 8, 12)


aic error: -1


05/15/2019 11:26:35 PM - INFO - SARIMAX: (7, 0, 8) x (6, 1, 8, 12)


aic error: -1


  1%|          | 3/500 [10:59<35:25:30, 256.60s/it]05/15/2019 11:32:24 PM - INFO - SARIMAX: (3, 0, 4) x (6, 0, 0, 12)


In [ ]:
score_table.to_csv(file_name2, sep=',', encoding='utf-8', mode='a', header=True)

# sequential code

In [7]:
"""Entry point to evolving the neural network. Start here."""
from __future__ import print_function
import logging
from optimizer import Optimizer
from tqdm import tqdm
import sys

# Setup logging.
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO#,
    #filename='log.txt'
)

def train_networks(networks, dataset, type_ser):
    """Train each network.

    Args:
        networks (list): Current population of networks
        dataset (str): Dataset to use for training/evaluating
    """
    pbar = tqdm(total=len(networks))
    for network in networks:
        network.train(dataset, type_ser)
        pbar.update(1)
    pbar.close()

def get_average_accuracy(networks):
    """Get the average accuracy for a group of networks.

    Args:
        networks (list): List of networks

    Returns:
        float: The average accuracy of a population of networks.

    """
    total_accuracy = 0
    for network in networks:
        if network.accuracy != -1 and network.accuracy != None:
            total_accuracy += network.accuracy
        else:
            continue

    return total_accuracy / len(networks)

def get_min_mse(networks):
    """Get the average accuracy for a group of networks/genomes.

    Args:
        networks (list): List of networks/genomes

    Returns:
        float: The average accuracy of a population of networks/genomes.

    """
    min_accuracy = 1000000
    min_network  = None
    for network in networks:
        if network.accuracy != -1 and network.accuracy < min_accuracy:
            min_accuracy = network.accuracy
            min_network=network
        else:
            continue
    return min_accuracy,min_network

def generate(generations, population, k, nn_param_choices, dataset, type_ser):
    """Generate a network with the genetic algorithm.

    Args:
        generations (int): Number of times to evole the population
        population (int): Number of networks in each generation
        nn_param_choices (dict): Parameter choices for networks
        dataset (str): Dataset to use for training/evaluating

    """
    logging.info("***generate(generations, population, nn_param_choices, dataset)***")
    optimizer = Optimizer(nn_param_choices)
    #print("--1--")
    networks = optimizer.create_population(population)
    #print("--2--")
    min_networks=[]
    # Evolve the generation.
    for i in range(generations):
        logging.info("*** Now in generation %d of %d ***" %(i + 1, generations))
        #print_networks(networks)
        # Train and get accuracy for networks.
        train_networks(networks, dataset, type_ser)

        # Get the average accuracy for this generation.
        average_accuracy = get_average_accuracy(networks)

        min_mse,net =get_min_mse(networks)
        #print_networks(net)
        #net.print_network()
        # Print out the average accuracy each generation.
        logging.info("Generation average: %.2f" % (average_accuracy ))#* 100
        logging.info('-'*80) #-----------
        logging.info("Generation min_mse: %.2f" % (min_mse ))#* 100
        logging.info('-'*80) #-----------

        # Evolve, except on the last iteration.
        if i != generations - 1:
            # Do the evolution.
            networks = optimizer.evolve(networks)
        min_networks.append(net)
    # Sort our final population.
    # Sort our final population according to performance.
    networks=[x for x in networks if x.accuracy !=-1 and x.accuracy != None]
    networks = sorted(networks, key=lambda x: x.accuracy, reverse=False)
    print("Generation done")

    # Print out the top 5 networks.
    logging.info("top 5 networks Generation" )
    logging.info('='*80) #-----------
    print_networks(networks[:5])
    
    pd.DataFrame(networks[:5]).to_csv(file_network, sep=',', encoding='utf-8', mode='a', header=True)
    for n in range(0,5):
        score_table.loc[n+k,2:] = networks[n].param, networks[n].param_seasonal, networks[n].accuracy
    
    logging.info("min networks Generation" )
    logging.info('-'*80) #-----------
    min_networks = sorted(min_networks, key=lambda x: x.accuracy, reverse=False)
    print_networks(min_networks)
    return networks
    
def print_networks(networks):
    """Print a list of networks.

    Args:
        networks (list): The population of networks

    """
    logging.info('#'*80)
    for network in networks:
        network.print_network()

def main():
    """Evolve a network."""
    generations  = [20]#, 10, 20, 50, 100, 200]  # Number of times to evole the population.
    populations  = [10, 50, 100, 200, 300, 400, 500]  # Number of networks in each generation.
    dataset = 'cifar10'
    k=0
    nn_param_choices = {
        'p_values' : range(0, 12),#[0, 1, 4, 6, 8, 10],
        'd_values' : range(0, 2),#[0, 1, 2], #range(0, 3)
        'q_values' : range(0, 12),#[0, 1, 2], #range(0, 3)
        'sp_values': range(0, 12),#[0, 1, 4, 6, 8, 10],
        'sd_values': range(0, 2),#[0, 1, 2],
        'sq_values': range(0, 12) #[0, 1, 2]
    }
    type_ser ='normalized'
    #for type_ser in ['normal', 'log', 'loglog']: #'normalized',
    for generation in generations: #'normalized',
        for population in populations:
            print("$$$$$$$$$$$$$$$$$$$$ %s $$$$$$$$$$$$$$$$$"% (type_ser))
            logging.info("***Evolving %d generations with population %d***" % (generation, population))
            networks = generate(generation, population, k, nn_param_choices, dataset, type_ser)
            k=k+5
            clear_output()
    return networks

/usr/local/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
#if __name__ == '__main__':
networks = main()

In [ ]:
for i in range(0,len(networks[0:5])):
    print(networks[i].param, networks[i].param_seasonal, networks[i].accuracy)
    #score_table.loc[i,2:] = networks[i].param, networks[i].param_seasonal, networks[i].accuracy

In [ ]:
score_table.to_csv(file_name, sep=',', encoding='utf-8', mode='a', header=True)

https://github.com/khanhnamle1994/applied-machine-learning/blob/6fe5fbce5e1cc79eb65bcd4e92c646d53798356b/Time-Series-Forecasting/how_to_create_an_ARIMA_model_for_Time_Series_Forecasting_with_Python.ipynb
how_to_create_an_ARIMA_model_for_Time_Series_Forecasting_with_Python.ipynb

https://github.com/aspiringguru/ML_Time_series_methods2/blob/6221da5c01be197a7803559eb69e50c4696f3a2c/example_ARIMA_future_predict.ipynb

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas.tools.plotting import autocorrelation_plot
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
print ("libraries loaded")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
def norm_series(series):
    y = series.resample('MS').mean()
    values2 = y.values
    values2 = values2.reshape((len(values2), 1))
    # train the normalization
    scaler2 = MinMaxScaler(feature_range=(-1, 1))
    scaler2 = scaler2.fit(values2)
    ser2 = DataFrame(y)
    ser2['normalized']= scaler2.transform(values2)
    ser2['log']=np.log(values2 )
    ser2['loglog']=np.log(np.log(values2 ))
    ser2.head(4)
    return ser2, scaler2

In [ ]:
#  dealing missing value

def check_missing(df):
    # Explore missing data
    missing_data = df.isnull().sum()
    # print(missing_data.dtypes)
    # print(type(missing_data))
    total_data = len(df)
    df_missing_data = missing_data.to_frame()
    df_missing_data.columns = ['counts']
    # Identify missing categories
    df_missing_data = df_missing_data[df_missing_data.counts != 0]
    # Calculate missing percentage
    df_missing_data['missing_percent'] = (df_missing_data.counts / total_data)*100
    #print(df_missing_data)
    #print(len(df_missing_data))
    return df_missing_data


In [ ]:
from preprocessing import preprocess
import os
from os.path import isfile, join

path = './'
#filename1='C://Users/gad/Desktop/work practical/Weather Data/data/raw/CA/CA_1981-1985.txt'
#train_data = pd.read_csv('C://Users/gad/Desktop/work practical/Weather Data/data/raw/CA/CA_1981-1985.txt')
train =[]
train.append(preprocess(join(path, '433440-99999-merge.op')))
df=pd.concat(train)

In [ ]:
# Create list of variable types
all_variables=['STN', 'WBAN', 'YEARMODA', 'TEMP', 'TEMP_count', 'DEWP', 'DEWP_count', 'SLP', 'SLP_count', 'STP', 'STP_count',
               'VISIB', 'VISIB_count', 'WDSP', 'WDSP_count', 'MXSPD', 'GUST', 'MAX', 'MIN','PRCP', 'SNDP', 'FRSHTT', 'MAX_flag',
               'MIN_flag', 'PRCP_A', 'PRCP_B', 'PRCP_C', 'PRCP_D', 'PRCP_E', 'PRCP_F', 'PRCP_G', 'PRCP_H','PRCP_I']

selected_list=['STN', 'WBAN', 'YEARMODA','TEMP', 'DEWP', 'SLP', 'STP', 'VISIB', 'WDSP', 'MXSPD', 'GUST', 'MAX', 'MIN',
               'PRCP', 'SNDP']# 15 columns we need to analysis for correlation 

#missing_list = ['DEWP','SLP','STP','VISIB','WDSP','MXSPD','GUST','MAX','MIN','PRCP','SNDP']#all columns contains missing values

#drop_list=['GUST','SNDP']#drop each column : where missing value more than 95%

#code to drop columns which missing values >90 %
miss_col=check_missing(df)
drop_col_list=[]
for i in miss_col.index:
    if miss_col.missing_percent[i]>90:
        drop_col_list.append(i)

# 13 columns we need to fill missing data
# difference between after_drop_list= selected_list-drop_col_list
#after_drop_list=list(set(selected_list)-set(drop_col_list))
s = set(drop_col_list)
after_drop_list = [x for x in selected_list if x not in s]

# 10 columns we need to train/test data
# difference between after_drop_list= selected_list-drop_col_list
#train_list=list(set(after_drop_list)-set(['STN', 'WBAN', 'YEARMODA']))
s = set(['STN', 'WBAN', 'YEARMODA'])
train_list = [x for x in after_drop_list if x not in s]
     
# Preprocess data #
selected_list2=['YEARMODA','TEMP']
df=df[selected_list]
df_after_drop =df[after_drop_list]
#df_after_drop.to_csv('df_after_drop.csv')
df_after_drop2 =df[selected_list2]
df_after_drop.head(3)

In [ ]:
df_after_drop2.to_csv("433440-99999-merge.csv",index=False)

In [ ]:
def parser(x):
	return datetime.strptime(x, '%Y-%m-%d')

# Get the data.
filename='433440-99999-merge.csv'

#filename='india_all_stations_comma.txt'
series = read_csv( filename, header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
    
series=series.dropna(axis=0)
#series = series.iloc[:1000,]  # independent variables
print(series.head())
series.describe()

In [ ]:
series2,scaler2 =norm_series(series)
series2.head(3)

In [ ]:
for i in ['TEMP' ,'normalized','log','loglog']:
    series2[i].plot(figsize=(12,8))
    plt.title(i,fontsize=24)
    pyplot.show()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(series2['TEMP'].values.squeeze(), lags=12, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(series2['TEMP'], lags=12, ax=ax2)

In [ ]:
#series2.plot()
pyplot.show()

In [ ]:
for i in ['TEMP' ,'normalized','log','loglog']:
    plt.title(i,fontsize=24)
    autocorrelation_plot(series2[i])
    pyplot.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
plot_acf(series2['TEMP'])
for i in ['TEMP' ,'normalized','log','loglog']:
    plot_acf(series2[i])
    plt.title(i,fontsize=24)
    pyplot.show()

In [ ]:
#plot_pacf(series2['normalized'], lags=12)
for i in ['TEMP' ,'normalized','log','loglog']:
    plot_pacf(series2[i], lags=12)
    plt.title(i,fontsize=24)
    pyplot.show()

In [ ]:
from pandas import DataFrame
from scipy import stats

residuals8 = DataFrame(series2['normalized'])
residuals8.plot(kind='line')
plt.show()

In [ ]:
#arma_mod20 = sm.tsa.ARMA(series, (5,1,0)).fit(disp=False)
param , param_seasonal=[(0,1,5),(3,1,3,12)]
mod = sm.tsa.statespace.SARIMAX(series2['normalized'], order=param, seasonal_order=param_seasonal,
                                            enforce_stationarity=False, enforce_invertibility=False)
#arma_mod20 = sm.tsa.ARMA(series2['normalized'], (1,1,2)).fit(disp=False)
#print(arma_mod20.params)
arma_mod20 = mod.fit()
print(arma_mod20.aic, arma_mod20.bic, arma_mod20.hqic)
print(arma_mod20.summary())

In [ ]:
pred = arma_mod20.get_prediction(start=pd.to_datetime('2000-01-01'), dynamic=False)
y_forecasted = pred.predicted_mean
#print ("type(y_forecasted):", type(y_forecasted))
y_truth = series2['normalized']['2000-01-01':]

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
error = np.sqrt(mean_squared_error(y_forecasted , y_truth))

print('Test RMSE: %.3f' % error)
print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, arma_mod20.aic))
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 4)))

In [ ]:
predict_ci.iloc[-70:, 0]

In [ ]:
# In-sample one-step-ahead predictions, and out-of-sample forecasts
nforecast = 20
predict = arma_mod20.get_prediction(start=pd.to_datetime('2000-01-01'))
#idx = np.arange(len(predict.predicted_mean))
idx = predict.predicted_mean.index
predict_ci = predict.conf_int(alpha=0.05)

# Graph
fig, ax = plt.subplots(figsize=(12,6))
ax.xaxis.grid()
ax.plot(series2['normalized'], 'k.', label='Observations')
ax.fill_between(idx[-70:], predict_ci.iloc[-70:, 0], predict_ci.iloc[-70:, 1], alpha=0.1)

# Plot
ax.plot(idx[:-nforecast], predict.predicted_mean[:-nforecast], 'blue', label='One-step-ahead Prediction')
ax.plot(idx[-nforecast:], predict.predicted_mean[-nforecast:], 'r--', linestyle='--', linewidth=2, label='Forecast1')

ax.fill_between(idx[-70:], predict_ci.iloc[-70:, 0], predict_ci.iloc[-70:, 1], alpha=0.15)


#pred_means = pred_res.predicted_mean
forecast = arma_mod20.get_forecast('2020-01-01')
forecast.predicted_mean.plot(ax=ax, style='y', label='Forecast2')
forecast_ci = forecast.conf_int()
forecast_index = predict.predicted_mean.index[len(predict.predicted_mean.index)-37:] + len(forecast_ci)
ax.fill_between(forecast_index[:], forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], alpha=0.01)

ax.set(title='One-step-ahead Prediction');
# Cleanup the image
ax.set_ylim();
legend = ax.legend(loc='lower left');


In [ ]:
pred_res = arma_mod20.get_prediction(start=pd.to_datetime('2000-01-01'), dynamic=False,alpha=0.05)
#pred_means = pred_res.predicted_mean
forecast = arma_mod20.get_forecast('2020-01-01')

    # Specify your prediction intervals by alpha parameter. alpha=0.05 implies 95% CI
pred_cis = pred_res.conf_int(alpha=0.05)
fig, ax = plt.subplots(figsize=(10,4))

# Plot the results
series2['normalized'].plot(ax=ax, style='k.', label='Observations')
pred_res.predicted_mean.plot(ax=ax, label='One-step-ahead Prediction')
predict_ci = pred_res.conf_int(alpha=0.05)
predict_index = np.arange(len(predict_ci))
#ax.fill_between(predict_index[-70:], predict_ci.iloc[-70:, 0], predict_ci.iloc[-70:, 1], alpha=0.1)

forecast.predicted_mean.plot(ax=ax, style='r', label='Forecast')
forecast_ci = forecast.conf_int()
forecast_index = predict.predicted_mean.index[len(predict.predicted_mean.index)-37:] + len(forecast_ci)
ax.fill_between(forecast_index[:], forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], alpha=0.01)
ax.set(title='One-step-ahead Prediction');
# Cleanup the image
ax.set_ylim();
legend = ax.legend(loc='lower left');

In [ ]:
# Specify your prediction intervals by alpha parameter. alpha=0.05 implies 95% CI
pred_cis = y_forecasted.conf_int(alpha=0.05)

    # You can then plot it (import matplotlib first)
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1,1,1)
    #Actual data
ax.plot(ser3['TEMP'].astype('float64'), '--', color="blue", label='data')
    # Means
ax.plot(y_forecasted, lw=1, color="black", alpha=0.5, label='SARIMAX')
ax.fill_between(pred_means.index, pred_cis.iloc[:, 0], pred_cis.iloc[:, 1], alpha=0.05)
ax.legend(loc='upper right')
plt.draw()

In [ ]:
y_forecasted

In [ ]:
ser3 = DataFrame(series2['normalized'])
ser3['TEMP'] = DataFrame(series2['TEMP'])

In [ ]:

ser3['inversed_pred'] = scaler2.inverse_transform(arma_mod20.resid.reshape((len(arma_mod20.resid), 1)))


In [ ]:
forecast1 = arma_mod20.forecast(9)
ser3['forecast1'] =forecast1
ser3.head()

In [ ]:
sm.stats.durbin_watson(arma_mod20.resid.values)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
#ax = arma_mod20.resid.plot(ax=ax)

ax = ser3['inversed_pred'].plot(ax=ax)

In [ ]:
resid = arma_mod20.resid
stats.normaltest(resid)

In [ ]:
from statsmodels.graphics.api import qqplot

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
fig = qqplot(resid, line='q', ax=ax, fit=True)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid, lags=40, ax=ax2)

In [ ]:
r,q,p = sm.tsa.acf(resid.values.squeeze(), qstat=True)
data = np.c_[range(1,41), r[1:], q, p]
table = pd.DataFrame(data, columns=['lag', "AC", "Q", "Prob(>Q)"])
print(table.set_index('lag'))

In [ ]:
predict_sunspots = arma_mod20.predict('2012-06-01', '2016-06-01', dynamic=True)
print(predict_sunspots)

In [ ]:
forecast = arma_mod20.get_forecast(5, exog='2016-05-01')
print(forecast.conf_int())
print(forecast)
arma_mod20.plot_diagnostics(figsize=(10,8))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,8))
fig = arma_mod20.plot_predict(start='2012-06-01', end='2016-05-01',  dynamic=False, ax=ax)
legend = ax.legend(loc='upper left')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

from scipy import stats
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot

from statsmodels.tsa.stattools import adfuller

# function to calculate MAE, RMSE
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.api import qqplot


s_test = adfuller(series2['normalized'], autolag='AIC')
# extract p value from test results
print ("p value > 0.05 means data is non-stationary: ", s_test[1])


In [ ]:
s_test

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 10}

matplotlib.rc('font', **font)
decomposition = seasonal_decompose(series2['normalized'])

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize=(17,9))
plt.subplot(411)
plt.title('Time Series - Decomposed')
plt.plot(ser3['TEMP'], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(18,8))
ts_predict = arma_mod20.predict('2015-06-01', '2016-06-01')
plt.plot(ser3['TEMP'], label='Actual')
#plt.plot(scaler2.inverse_transform(ts_predict).reshape(-1, 1), 'r--', label='Predicted')
plt.plot(ser3['inversed_pred'], 'r--', label='Predicted')
plt.xlabel('Year-Month',)
plt.ylabel('TEMP')
plt.legend(loc='best')

In [ ]:
#fig, ax = plt.subplots(figsize=(12, 8))
#ax = ser3['TEMP'].loc['2010':].plot(ax=ax)
#fig = scaler2.inverse_transform(arma_mod20.plot_predict('2015-06-01', '2016-06-01', dynamic=False, ax=ax, plot_insample=True))
ser3[['TEMP', 'inversed_pred']].plot(figsize=(12, 8))

In [ ]:
npredict =ser3['TEMP'].shape[0]
fig, ax = plt.subplots(figsize=(12,6))
npre = 12
ax.set(title='TEMP', xlabel='Date', ylabel='normalized')
ax.plot(ser3.index[-npredict-npre+1:], ser3.ix[-npredict-npre+1:, 'TEMP'], 'o', label='Observed')
ax.plot(ser3.index[-npredict-npre+1:], ser3.ix[-npredict-npre+1:, 'inversed_pred'], 'g', label='Dynamic forecast')
legend = ax.legend(loc='lower right')
legend.get_frame().set_facecolor('w')

In [ ]:
def mean_forecast_err(y, yhat):
    return y.sub(yhat).mean()

In [ ]:
mean_forecast_err(series2['normalized'], predict_sunspots)

In [ ]:
print(sm.tsa.adfuller(series2['normalized'])[1])

In [ ]:
len(series2['normalized'])

In [ ]:
fig = sm.graphics.tsa.month_plot(series2['TEMP'])

In [ ]:
from pandas import DataFrame
# fit model
model = ARIMA(series2['normalized'], order=(12,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())

In [ ]:
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA
import numpy
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [ ]:
# seasonal difference
X = series2['normalized'].values
days_in_year = 1
differenced = difference(X, days_in_year)
differenced

In [ ]:
arima_order=(12,1,0)

size = int(len(differenced) * 0.66)
train, test = differenced[0:size], differenced[size:len(differenced)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=arima_order)
    model_fit = model.fit(disp=0)
    yhat = model_fit.forecast()[0]
        
    # invert the differenced forecast to something usable
    yhat = inverse_difference(history, yhat, 1)
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = np.sqrt(mean_squared_error(test, predictions))
MAPE = sum(abs(predictions / test) / len(test))
print('Test RMSE: %.3f' % error)
#print('Test MAPE: %.3f' % MAPE)
MAPE

In [ ]:
X = series2['normalized'].values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
error2 = np.sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % error2)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
series[:3]

In [ ]:
# The 'MS' string groups the data in buckets by start of the month
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
#explanation of grouping codes.
#https://chrisalbon.com/python/pandas_group_data_by_time.html
#.resample('MS') = group by month start
y = series2['normalized'].resample('MS').mean()
print (type(y))
# The term bfill means that we use the value before filling in missing values
y = y.fillna(y.bfill())

print(y.head())

In [ ]:
y.plot(figsize=(15, 6))
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.plot.html
plt.show()

In [ ]:
y

In [ ]:
#time-series seasonal decomposition
#NB: this is useful in this data due to the seasonal effects, not all data will show seasonal effects.

from pylab import rcParams
#make plot larger for readability.
rcParams['figure.figsize'] = 11, 9

decomposition = sm.tsa.seasonal_decompose(y, freq=12, model='additive')
fig = decomposition.plot()
plt.show()
print ("plotted")
#https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-visualization-with-python-3

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))
print (type(pdq), len(pdq), pdq)

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
X = series2['normalized'].values
days_in_year = 1
differenced = difference(y, days_in_year)
differenced

In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

#http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html

minResult = []
minParam = []
minParam_Seasonal = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y, order=param, seasonal_order=param_seasonal,
                                            enforce_stationarity=False, enforce_invertibility=False)

            results = mod.fit()
            minResult.append( results.aic )
            minParam.append(param)
            minParam_Seasonal.append(param_seasonal)
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            #NB: some parameter combinations might result in errors. catch errors and continue.
            continue
print ("end-----")
print (min(minResult))

In [ ]:
index = minResult.index(min(minResult))
print (index)

print ("Minimum AIC value ", min(minResult), " occurs with SARIMAX(", minParam[index], ", ", minParam_Seasonal[index], ")" )

In [ ]:
#redo the model using the optimum input parameters.
mod = sm.tsa.statespace.SARIMAX(y, order=(minParam[index]), seasonal_order=(minParam_Seasonal[index]),
                                enforce_stationarity=False, enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])
print(results.summary())

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
#http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAXResults.plot_diagnostics.html
#http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAXResults.html
plt.show()

In [ ]:
#forecasts start 1st January 1998.
pred = results.get_prediction(start=pd.to_datetime('2013-01-01'), dynamic=False)
pred_ci = pred.conf_int()

In [ ]:
ax = y['2000':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index, pred_ci.iloc[:, 0],pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('TEMP')
plt.legend()

plt.show()

In [ ]:
y_forecasted = pred.predicted_mean
print ("type(y_forecasted):", type(y_forecasted))
y_truth = y['2013-01-01':]

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print (type(mse))
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
#if mean squared error were zero, prediction would be perfect.

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('2013-01-01'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()
print ("type(pred_dynamic):", type(pred_dynamic))
print ("type(pred_dynamic_ci):", type(pred_dynamic_ci), pred_dynamic_ci.shape)
print (list(pred_dynamic_ci))

In [ ]:
print (type(y), y.shape)
y.head()

In [ ]:
ax = y['2000':].plot(label='observed', figsize=(20, 15))
#figsize=(20, 15) sets plot dimensions
#

pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,pred_dynamic_ci.iloc[:, 0],pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('2013-01-01'), y.index[-1],alpha=.1, zorder=-1)

ax.set_xlabel('Date')
ax.set_ylabel('TEMP')

plt.legend()
plt.show()

In [ ]:
# Extract the predicted and true values of our time series
y_forecasted = pred_dynamic.predicted_mean
y_truth = y['2013-01-01':]

print ("type(y_forecasted):", type(y_forecasted))
print ("type(y_truth):", type(y_truth))

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

In [ ]:
# Get forecast 500 steps ahead in future
pred_uc = results.get_forecast(steps=50)
print ("type(pred_uc):", type(pred_uc))

# Get confidence intervals of forecasts
#http://www.statsmodels.org/dev/dev/generated/statsmodels.base.model.GenericLikelihoodModelResults.conf_int.html
pred_ci = pred_uc.conf_int()
print ("type(pred_ci):", type(pred_ci), pred_ci.shape)
print ("pred_ci col names:", list(pred_ci))

In [ ]:
pred_uc.predicted_mean

In [ ]:
ax = y.plot(label='observed', figsize=(20, 15))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,pred_ci.iloc[:, 0],pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('TEMP')

plt.legend()
plt.show()

In [ ]:
start_index = '2000-01-01'
end_index   = '2016-12-01'
forecast = results.predict(start=start_index, end=end_index)
series2['forecast']=forecast
# inverse transform and print the first 5 rows
series2['inversed_forecast'] = scaler2.inverse_transform(series2['forecast'].reshape(-1, 1))

In [ ]:
series2

In [ ]:
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA
import numpy

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# load dataset
# Get the data.
filename='433440-99999-merge.csv'

#filename='india_all_stations_comma.txt'
series = read_csv( filename, header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
#series = Series.from_csv('dataset.csv', header=None)
# seasonal difference
X = series.values
days_in_year = 365
differenced = difference(X, days_in_year)
# fit model
model = ARIMA(differenced, order=(7,0,1))
model_fit = model.fit(disp=0)
# multi-step out-of-sample forecast
forecast = model_fit.forecast(steps=7)[0]
# invert the differenced forecast to something usable
history = [x for x in X]
day = 1
print("Running the example prints the forecast for the next 7 days.")
for yhat in forecast:
	inverted = inverse_difference(history, yhat, days_in_year)
	print('Day %d: %f' % (day, inverted))
	history.append(inverted)
	day += 1

In [ ]:
# seasonal difference
X = series.values
days_in_year = 365
differenced = difference(X, days_in_year)
# fit model
model = ARIMA(differenced, order=(7,0,1))
model_fit = model.fit(disp=0)
# multi-step out-of-sample forecast
start_index = len(differenced)
end_index = start_index + 6
forecast = model_fit.predict(start=start_index, end=end_index)
# invert the differenced forecast to something usable
history = [x for x in X]
day = 1
for yhat in forecast:
	inverted = inverse_difference(history, yhat, days_in_year)
	print('Day %d: %f' % (day, inverted))
	history.append(inverted)
	day += 1

https://github.com/georgiev2592/data_science_final_project/blob/cbe75508a7bb6a5e46a82276aaac735854f61480/05.%20Machine%20Learning%20-%20Configure%20ARIMA.ipynb

In [ ]:
import pandas as pd
import numpy as np

import warnings

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# create a differenced series
def difference(data, interval=1):
    diff = list()
    for i in range(interval, len(data)):
        value = data[i] - data[i - interval]
        diff.append(value)
    return np.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [ ]:
#y
X = y.values
#X

In [ ]:
# seasonal difference
days_in_year = 1
differenced = difference(X, days_in_year)
differenced

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = len(X) - 7
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    
    # make predictions
    predictions = []
    
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        
        # invert the differenced forecast to something usable
        yhat = inverse_difference(history, yhat, 1)
        
        predictions.append(yhat)
        history.append(test[t])
    
    # calculate out of sample error
    error = np.sqrt(mean_squared_error(test, predictions))
    
    return error
#----------------------------------------------------------------------------
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order, rmse))
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:
# evaluate parameters
p_values = range(0, 10)
d_values = range(0, 10)
q_values = range(0, 10)

warnings.filterwarnings("ignore")

evaluate_models(differenced, p_values, d_values, q_values)

In [ ]:
## https://www.statsmodels.org/dev/examples/notebooks/generated/tsa_filters.html

%matplotlib inline

from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm

In [ ]:
index = pd.Index(sm.tsa.datetools.dates_from_range('2000Q1', '2016Q3'))
print(index)

In [ ]:
filename='india_all_stations_label.csv'

#filename='india_all_stations_comma.txt'
series = read_csv( filename, header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [ ]:
series

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
series.TEMP.plot(ax=ax);
legend = ax.legend(loc = 'upper left');
legend.prop.set_size(20);

In [ ]:
gdp_cycle, gdp_trend = sm.tsa.filters.hpfilter(series.values)

In [ ]:
gdp_decomp = DataFrame(series[:].copy())
gdp_decomp

In [ ]:
gdp_decomp['cycle'] = gdp_cycle
gdp_decomp['trend'] = gdp_trend

In [ ]:
gdp_decomp

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
gdp_decomp[["TEMP","trend"]]["2010-03-31":].plot(ax=ax, fontsize=16);
legend = ax.get_legend()
legend.prop.set_size(20);

In [ ]:
bk_cycles = sm.tsa.filters.bkfilter(series[["TEMP","MIN"]])
#TEMP 	DEWP 	STP 	VISIB 	WDSP 	MXSPD 	MAX 	MIN

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111)
bk_cycles.plot(ax=ax, style=['r--', 'b-']);

In [ ]:
print(sm.tsa.stattools.adfuller(series["TEMP"])[:3])

In [ ]:
print(sm.tsa.stattools.adfuller(series["MIN"])[:3])

In [ ]:
cf_cycles, cf_trend = sm.tsa.filters.cffilter(dta[["TEMP","MIN"]])
print(cf_cycles.head(10))